In [5]:
# Usuals imports 
import numpy as np
import pandas as pd 
import requests

## Cities 

In [6]:
cities = ["Mont Saint Michel", "St Malo", "Bayeux", "Le Havre", "Rouen", "Paris", "Amiens", "Lille", "Strasbourg", "Chateau du Haut Koenigsbourg", "Colmar", "Eguisheim", "Besancon", "Dijon", "Annecy", "Grenoble", "Lyon", "Gorges du Verdon", "Bormes les Mimosas", "Cassis", "Marseille", "Aix en Provence", "Avignon", "Uzes", "Nimes", "Aigues Mortes", "Saintes Maries de la mer", "Collioure", "Carcassonne", "Ariege", "Toulouse", "Montauban", "Biarritz", "Bayonne", "La Rochelle"]

## Dataframe Cities 

In [7]:
cities_locations = pd.DataFrame(columns= ['city', 'country', 'latitude', 'longitude', 'adress'])

for city in cities:
    country = 'France'
    latitude = 0.0
    longitude = 0.0
    adress = ''

    # Creating urls params
    payload = {'city': city, 'country': country, 'format': 'jsonv2'}

    # Create the requests 
    r = requests.get('https://nominatim.openstreetmap.org/search.php?', params=payload)

    # check result available 
    if r.status_code == 200 :
        r_result = pd.DataFrame(r.json())
        #attrib location 
        try: 
            latitude = r_result['lat'][0]
            longitude = r_result['lon'][0]
            adress =  r_result['display_name'][0]

        except:
            #try with county 
            payload = {'county': city, 'country': country, 'format': 'jsonv2'}
            r = requests.get('https://nominatim.openstreetmap.org/search.php?', params=payload)
            r_result = pd.DataFrame(r.json())

            try: 
                latitude = r_result['lat'][0]
                longitude = r_result['lon'][0]
                adress =  r_result['display_name'][0]

            except:
                #try with street 
                payload = {'street': city, 'country': country, 'format': 'jsonv2'}
                r = requests.get('https://nominatim.openstreetmap.org/search.php?', params=payload)
                r_result = pd.DataFrame(r.json())

                try: 
                    latitude = r_result['lat'][0]
                    longitude = r_result['lon'][0]
                    adress =  r_result['display_name'][0]
                
                except:
                    pass

        new_city = pd.DataFrame(columns= ['city', 'country', 'latitude', 'longitude', 'adress'], 
                                   data= [[ city, country, latitude, longitude, adress]])

        cities_locations = pd.concat([cities_locations, new_city])

# reset index due to concat.
cities_locations.reset_index(inplace= True)
cities_locations.drop(columns=['index'], inplace= True)

## Dataframe Wheater in Cities :

In [37]:
# https://www.weatherbit.io/api
# https://www.weatherbit.io/api/weather-current
# https://www.weatherbit.io/api/codes

KEYS = open('src/KEYS.txt', 'r').read()


# cities_locations = cities_locations.assign(temperature=0, app_temp=0, C="")

# len(cities_locations)
for item in range(1):
    latitude = cities_locations['latitude'][item]
    longitude = cities_locations['longitude'][item]

    # Creating urls params
    payload = {'key': KEYS, 'lat': latitude, 'lon': longitude, }

    # Create the requests 
    r = requests.get('http://api.weatherbit.io/v2.0/current?', params=payload)

    # check result available 
    if r.status_code == 200 :
        r_result = pd.DataFrame(r.json())
        data = r_result['data'][0]
        weather = data['weather']

        pressure = data['pres'] # Pressure (mb)
        temperature = data['temp'] # Temperature (c°)
        felt_temperature = data['app_temp'] # Apparent temperature (c°)
        humidity = data['rh'] # Relative humidity (%)
        cloud_coverage = data['clouds'] # Cloud coverage (%)
        dew_point = data['dewpt'] # Dew point (c°).
        wind = data['wind_spd'] # Wind speed (m/s).


        print(data) 


# weather: { 
    # code:Weather code.
    # description: Text weather description.



{'wind_cdir': 'NW', 'rh': 81, 'pod': 'd', 'lon': -1.5115, 'pres': 1016.96, 'timezone': 'Europe/Paris', 'ob_time': '2022-08-25 18:04', 'country_code': 'FR', 'clouds': 79, 'ts': 1661450660, 'solar_rad': 70.4, 'state_code': '28', 'city_name': 'Le Mont-Saint-Michel', 'wind_spd': 5.6136, 'slp': 1019.13, 'wind_cdir_full': 'northwest', 'sunrise': '05:16', 'app_temp': 19.2, 'dni': 441.63, 'vis': 16, 'sources': ['analysis'], 'h_angle': 64.3, 'dewpt': 15.8, 'snow': 0, 'aqi': 42, 'dhi': 49.3, 'wind_dir': 324, 'elev_angle': 9.19, 'ghi': 111.36, 'precip': 0, 'sunset': '19:04', 'lat': 48.636, 'uv': 0.820866, 'datetime': '2022-08-25:17', 'temp': 19.1, 'weather': {'icon': 'c03d', 'code': 803, 'description': 'Broken clouds'}, 'station': 'LFRD'}


## Goals 🎯

As the project has just started, your team doesn't have any data that can be used to create this application. Therefore, your job will be to: 

* √ Scrape data from destinations 
* √ Get weather data from each destination 
* Get hotels' info about each destination
* Store all the information above in a data lake
* Extract, transform and load cleaned data from your datalake to a data warehouse